In [ ]:
import pathlib
import numpy as np

from aiida import orm, load_profile
from aiida.engine import submit

from qe_tools import CONSTANTS

from aiida_quantumespresso_hp.workflows.hubbard import SelfConsistentHubbardWorkChain
from aiida_quantumespresso.data.hubbard_structure import HubbardStructureData

load_profile()

# BaTiO3
Let's define the BTO structure to use as example for this tutorial.

In [ ]:
cell = [
    [5.34198, -3.08419, 4.38073],
    [0.00000,  6.16839, 4.38073],
    [-5.3419, -3.08419, 4.38073],
]
cell = CONSTANTS.bohr_to_ang*np.array(cell)

structure = orm.StructureData(cell=cell)

structure.append_atom(position=np.dot([0.48674, 0.4867, 0.4867], cell),  symbols='Ti')
structure.append_atom(position=np.dot([0.51163, 0.5116, 0.0192], cell),  symbols='O' )
structure.append_atom(position=np.dot([0.51163, 0.0192, 0.5116], cell),  symbols='O' )
structure.append_atom(position=np.dot([0.01929, 0.5116, 0.5116], cell),  symbols='O' )
structure.append_atom(position=np.dot([0.99899, 0.9989, 0.9989], cell),  symbols='Ba')

## HubbardStructureData initialization
Let's initialize the HubbardStructureData with the BTO structure!

> Important: You only need to do these steps once! Later just keep track of this testing structure via the PK and reload it from the database so you don't get 100 BaTiO3 strucures in your storage. ^^

In [ ]:
hubbard_data = HubbardStructureData(structure=structure)

In [ ]:
hubbard_data

## Initializing the on-site Hubbard
Let's initialize the on-site Hubbard parameter for the titanium atom.

In [ ]:
hubbard_data.initialize_onsites_hubbard('Ti', '3d', 5.0) # index 0 for Ti ==> Hubbard U

Here how it is stored in the class.

In [ ]:
hubbard_data.hubbard_parameters

## Initializing the inter-site Hubbard
Let's initialize the inter-site Hubbard parameter for the titanium and oxygen atoms.

In [ ]:
hubbard_data.initialize_intersites_hubbard('Ti', '3d', 'O', '2p', 0.0001, number_of_neighbours=7)

The parameters are saved in the property `hubbard_parameters` as a list.

In [ ]:
hubbard_data.hubbard_parameters

In [ ]:
hubbard_data.store()

In [ ]:
hubbard_data.pk

## Running the `SelfConsistentHubbardWorkChain`

In [ ]:
hubbard_data = orm.load_node(2502)  # I load the node from the database instead of always regenerating it
print(hubbard_data.get_quantum_espresso_hubbard_card())

In [ ]:
pw_code = orm.load_code('qe-dev-pw@lumi-small')
hp_code = orm.load_code('qe-dev-hp@lumi-small')

builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data,
    protocol='fast',
    overrides=pathlib.Path('hubbard_overrides.yaml')
)

builder.skip_first_relax = True

In [ ]:
third_submit = submit(builder)
print(third_submit)

In [ ]:
first_submit_pk = 2234
second_submit_pk = 2448
third_submit_pk = 2642

In [ ]:
%%bash
verdi process show 2234
printf '=%.0s' {1..100}
echo
verdi process status 2234
printf '=%.0s' {1..100}
echo
verdi process report 2234
printf '=%.0s' {1..100}
echo

In [ ]:
%%bash
verdi process show 2448
printf '=%.0s' {1..100}
echo
verdi process status 2448
printf '=%.0s' {1..100}
echo
verdi process report 2448
printf '=%.0s' {1..100}
echo

In [ ]:
%%bash
verdi process show 2642
printf '=%.0s' {1..100}
echo
verdi process status 2642
printf '=%.0s' {1..100}
echo
verdi process report 2642
printf '=%.0s' {1..100}
echo